In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2024/sample_submission.json
/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json


In [2]:
import json

training_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json'
evaluation_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json'
evaluation_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json'
sample_submission_path = '/kaggle/input/arc-prize-2024/sample_submission.json'
training_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json'
test_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'

#function to load JSON data
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

#load each dataset
training_solutions = load_json_data(training_solutions_path)
evaluation_solutions = load_json_data(evaluation_solutions_path)
evaluation_challenges = load_json_data(evaluation_challenges_path)
sample_submission = load_json_data(sample_submission_path)
training_challenges = load_json_data(training_challenges_path)
test_challenges = load_json_data(test_challenges_path)

In [3]:
#inspect the structure of the data
def inspect_data(data, name):
    print(f"Inspecting {name}:")
    if isinstance(data, list):
        print(f"Number of items: {len(data)}")
        if len(data) > 0:
            print(f"Example item: {data[0]}")
    elif isinstance(data, dict):
        print(f"Number of keys: {len(data.keys())}")
        if len(data.keys()) > 0:
            first_key = list(data.keys())[0]
            print(f"Example item under key '{first_key}': {data[first_key]}")
    else:
        print("Unknown data type")
    print("\n")

inspect_data(training_solutions, "Training Solutions")
inspect_data(evaluation_solutions, "Evaluation Solutions")
inspect_data(evaluation_challenges, "Evaluation Challenges")
inspect_data(sample_submission, "Sample Submission")
inspect_data(training_challenges, "Training Challenges")
inspect_data(test_challenges, "Test Challenges")

Inspecting Training Solutions:
Number of keys: 400
Example item under key '007bbfb7': [[[7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 7, 0, 7, 7, 0, 0, 0, 0]]]


Inspecting Evaluation Solutions:
Number of keys: 400
Example item under key '00576224': [[[3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8], [2, 3, 2, 3, 2, 3], [8, 7, 8, 7, 8, 7], [3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8]]]


Inspecting Evaluation Challenges:
Number of keys: 400
Example item under key '00576224': {'test': [{'input': [[3, 2], [7, 8]]}], 'train': [{'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}, {'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 

In [4]:
def get_max_grid_size(challenges, solutions):
    max_input_height, max_input_width = 0, 0
    max_output_height, max_output_width = 0, 0
    
    for key in challenges.keys():
        challenge = challenges[key]
        for example in challenge['train']:
            input_grid = example['input']
            output_grid = example['output']
            max_input_height = max(max_input_height, len(input_grid))
            max_input_width = max(max_input_width, len(input_grid[0]))
            max_output_height = max(max_output_height, len(output_grid))
            max_output_width = max(max_output_width, len(output_grid[0]))
        for test_case in challenge['test']:
            test_input = test_case['input']
            max_input_height = max(max_input_height, len(test_input))
            max_input_width = max(max_input_width, len(test_input[0]))
            #assuming test_output size can be derived similarly

    return max_input_height, max_input_width, max_output_height, max_output_width

max_input_height, max_input_width, max_output_height, max_output_width = get_max_grid_size(training_challenges, training_solutions)
print(f"Max input size: {max_input_height}x{max_input_width}, Max output size: {max_output_height}x{max_output_width}")

Max input size: 30x30, Max output size: 30x30


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ARCDataset(Dataset):
    def __init__(self, challenges, solutions, max_size, transform=None):
        self.data = []
        self.max_size = max_size
        self.transform = transform
        for key in challenges.keys():
            challenge = challenges[key]
            solution = solutions[key]
            for example in challenge['train']:
                input_grid = example['input']
                output_grid = example['output']
                self.data.append((input_grid, output_grid))
            for test_case in challenge['test']:
                test_input = test_case['input']
                #use the corresponding solution as the target output
                test_output = solution[len(self.data) % len(solution)]
                self.data.append((test_input, test_output))

    def __len__(self):
        return len(self.data)

    def pad_grid(self, grid):
        padded_grid = np.zeros(self.max_size)
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                padded_grid[i][j] = grid[i][j]
        return padded_grid

    def __getitem__(self, idx):
        input_grid, output_grid = self.data[idx]
        input_grid = self.pad_grid(input_grid)
        output_grid = self.pad_grid(output_grid)
        input_grid = torch.tensor(input_grid, dtype=torch.float32).unsqueeze(0)
        output_grid = torch.tensor(output_grid, dtype=torch.float32).unsqueeze(0)
        if self.transform:
            input_grid = self.transform(input_grid)
            output_grid = self.transform(output_grid)
        return input_grid, output_grid

#define the maximum size
max_size = (30, 30)

#create datasets with padding
transform = transforms.Compose([transforms.Lambda(lambda x: x)])  #no additional transform needed
training_dataset = ARCDataset(training_challenges, training_solutions, max_size, transform=transform)
evaluation_dataset = ARCDataset(evaluation_challenges, evaluation_solutions, max_size, transform=transform)

#create dataloaders
training_loader = DataLoader(training_dataset, batch_size=1, shuffle=True)
evaluation_loader = DataLoader(evaluation_dataset, batch_size=1, shuffle=False)

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#perception module
class PerceptionModule(nn.Module):
    def __init__(self, in_channels):
        super(PerceptionModule, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(32 * 7 * 7, 128)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  #flatten the tensor
        x = self.dropout(x)
        x = F.relu(self.fc(x))
        return x

In [7]:
#memory module
class MemoryModule(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(MemoryModule, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        lstm_output, _ = self.lstm(x)
        
        #ensure lstm_output has the correct shape
        if lstm_output.ndim == 2:  #handle the case when lstm_output is 2D
            lstm_output = lstm_output.unsqueeze(1)  #add a dummy dimension
        last_hidden_state = lstm_output[:, -1, :]

        x = F.relu(self.fc1(last_hidden_state))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [8]:
#creativity module
class CreativityModule(nn.Module):
    def __init__(self, input_size):
        super(CreativityModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [9]:
#reasoning module
class ReasoningModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(ReasoningModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [10]:
#compassionate module
class CompassionateModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(CompassionateModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [11]:
#bias detection and mitigation module
class BiasDetectionMitigationModule(nn.Module):
    def __init__(self, input_size):
        super(BiasDetectionMitigationModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, input_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [12]:
class Yambi(nn.Module):
    """
    Yambi is an AI model designed to solve abstract reasoning tasks by integrating various cognitive abilities. 
    It consists of six main modules:
    
    1. Perception Module: Processes and understands input grids using convolutional neural networks.
    2. Memory Module: Stores and recalls information about previously seen tasks using LSTM networks.
    3. Creativity Module: Generates creative solutions and explores novel approaches using a latent space.
    4. Reasoning Module: Applies logical reasoning to infer solutions based on perceived information.
    5. Compassionate Module: Analyzes emotional context and adjusts responses accordingly.
    6. Bias Detection and Mitigation Module: Detects and mitigates biases in the reasoning output.

    These modules work together to perceive, remember, create, reason, empathize, and ensure fairness in solving
    abstract reasoning tasks.
    """

    def __init__(self, perception_model, memory_model, creativity_model, reasoning_model, compassionate_model, bias_detection_mitigation_model):
        super(Yambi, self).__init__()
        self.perception_model = perception_model
        self.memory_model = memory_model
        self.creativity_model = creativity_model
        self.reasoning_model = reasoning_model
        self.compassionate_model = compassionate_model
        self.bias_detection_mitigation_model = bias_detection_mitigation_model
    
    def forward(self, x):
        # Perception Module
        perception_output = self.perception_model(x)

        # Memory Module
        memory_output = self.memory_model(perception_output)

        # Creativity Module
        creativity_output = self.creativity_model(memory_output)

        # Reasoning Module
        reasoning_output = self.reasoning_model(creativity_output)

        # Bias Detection and Mitigation Module
        unbiased_reasoning_output = self.bias_detection_mitigation_model(reasoning_output)

        # Compassionate Module
        compassionate_output = self.compassionate_model(unbiased_reasoning_output)

        return unbiased_reasoning_output, compassionate_output

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
#instantiate modules
perception_model = PerceptionModule(in_channels=1)
memory_model = MemoryModule(input_size=128, hidden_size=128, output_size=128, num_layers=2)
creativity_model = CreativityModule(input_size=128)
reasoning_model = ReasoningModule(input_size=128, output_size=128)
compassionate_model = CompassionateModule(input_size=128, output_size=10)
bias_detection_mitigation_model = BiasDetectionMitigationModule(input_size=128)

#move models to device
perception_model.to(device)
memory_model.to(device)
creativity_model.to(device)
reasoning_model.to(device)
compassionate_model.to(device)
bias_detection_mitigation_model.to(device)

#instantiate the Yambi model
yambi_model = Yambi(perception_model, memory_model, creativity_model, reasoning_model, compassionate_model, bias_detection_mitigation_model)
yambi_model.to(device)

Yambi(
  (perception_model): PerceptionModule(
    (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (dropout): Dropout(p=0.5, inplace=False)
    (fc): Linear(in_features=1568, out_features=128, bias=True)
  )
  (memory_model): MemoryModule(
    (lstm): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.2)
    (fc1): Linear(in_features=128, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=128, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (creativity_model): CreativityModule(
    (fc1): Linear(in_features=128, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
    (fc3): Linear(in_features=256, out_features=128, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (reasoning_model): ReasoningModu

In [15]:
#train the Yambi model
def train_model(yambi_model, train_loader, optimizer, criterion, device, num_epochs=10):
    yambi_model.train()  #set the yambi_model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            #zero the gradients
            optimizer.zero_grad()

            #forward pass
            unbiased_reasoning_output, compassionate_output = yambi_model(inputs)

            #compute the loss
            loss = criterion(unbiased_reasoning_output, labels)

            #backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [16]:
def calculate_arc_score(model, evaluation_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in evaluation_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).squeeze(1)

            reasoning_output, compassionate_output = model(inputs)
            predicted_output = torch.argmax(reasoning_output, dim=1)

            correct += (predicted_output == labels).sum().item()
            total += labels.numel()

    model.train() 
    
    return correct / total

#evaluate the model on the evaluation dataset
arc_score = calculate_arc_score(yambi_model, evaluation_loader, device)
print(f'ARC Score: {arc_score * 100:.2f}%')

ARC Score: 0.00%
